# Application of Graph Rewriting to Natural Language Processing

## Kapittel 1: Programming with graphs

### 1.1. creating a graph


In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ingeridd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ingeridd/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
g = dict()

In [3]:
g["W1"] = ("the", [])

In [4]:
g [ 'W2'] = ( 'child' , [ ] )
g [ 'W3'] = ( 'plays' , [ ] )
g [ 'W3'] [ 1 ] . append (( 'suj' , 'W2') )
g [ 'W2'] [ 1 ] . append (( 'det' , 'W1') )

In [5]:
g["W2"][0]


'child'

In [6]:
def add_node(graph, node, label): 
    graph[node] = (label, [])

def add_edge(graph, node, edge, head):
    if (edge, head) not in graph[node][1]:
        graph[node][1].append((edge, head))

In [7]:
add_node(g, "W4", "the")
add_node(g, "W5", "fool")
add_edge(g, "W3", "obj", "W5")
add_edge(g, "W5", "det", "W4")


In [8]:
g

{'W1': ('the', []),
 'W2': ('child', [('det', 'W1')]),
 'W3': ('plays', [('suj', 'W2'), ('obj', 'W5')]),
 'W4': ('the', []),
 'W5': ('fool', [('det', 'W4')])}

In [9]:
def construct_flat_graph(sentence):
    word_list = nltk.word_tokenize(sentence)
    word_graph = dict()
    for i in range(len(word_list)):
        add_node(word_graph, f'W{i}', word_list[i]) 
    for i in range(len(word_list) - 1): 
        add_edge(word_graph, f'W{i}', 'SUC', f'W{i+1}')
    return word_graph

word_graph = construct_flat_graph("She takes a glass")
word_graph

{'W0': ('She', [('SUC', 'W1')]),
 'W1': ('takes', [('SUC', 'W2')]),
 'W2': ('a', [('SUC', 'W3')]),
 'W3': ('glass', [])}

### Excerise 1.1
Finish constructing the following ﬂat graph so that there is a
'SUC*' edge between each word and one of its distant successors. 

In [10]:
for i in range(len(word_graph)-1): 
    for j in range(i+2, len(word_graph)): 
        add_edge(word_graph, f"W{i}", 'SUC*',  f"W{j}")
word_graph

{'W0': ('She', [('SUC', 'W1'), ('SUC*', 'W2'), ('SUC*', 'W3')]),
 'W1': ('takes', [('SUC', 'W2'), ('SUC*', 'W3')]),
 'W2': ('a', [('SUC', 'W3')]),
 'W3': ('glass', [])}


### EXERCISE 1.2.
Write a function to compute a graph in which all of the edges
have been reversed. 

In [11]:
def reverse_edges(graph):
    rev_g = dict()
    [add_node(rev_g,node,labels[0]) for node, labels in graph.items()]

    for node, (label, edges) in graph.items(): 
        if not edges:
            continue
        for pos, head in edges:
            add_edge(rev_g, head, pos, node)
    return rev_g
    
rev_g = reverse_edges(g)
rev_g

{'W1': ('the', [('det', 'W2')]),
 'W2': ('child', [('suj', 'W3')]),
 'W3': ('plays', []),
 'W4': ('the', [('det', 'W5')]),
 'W5': ('fool', [('obj', 'W3')])}

In [12]:
g

{'W1': ('the', []),
 'W2': ('child', [('det', 'W1')]),
 'W3': ('plays', [('suj', 'W2'), ('obj', 'W5')]),
 'W4': ('the', []),
 'W5': ('fool', [('det', 'W4')])}

## 1.2 Feature structures

In [13]:
features_plays = {'phon': 'plays','cat':'V'}

In [14]:
features_plays['cat']


'V'

In [15]:
g = dict()

add_node(g, 'W1', {'phon':'the','cat':'DET'})
add_node(g, 'W2', {'phon':'child','cat':'N'})
add_node(g, 'W3', {'phon':'plays','cat':'V'})
add_node(g, 'W4', {'phon':'the','cat':'DET'})
add_node(g, 'W5', {'phon':'fool','cat':'N'})
add_edge(g, 'W2', 'det', 'W1')
add_edge(g, 'W3', 'suj', 'W2')
add_edge(g, 'W3', 'obj', 'W5')
add_edge(g, 'W4', 'det', 'W4')

In [16]:
g


{'W1': ({'phon': 'the', 'cat': 'DET'}, []),
 'W2': ({'phon': 'child', 'cat': 'N'}, [('det', 'W1')]),
 'W3': ({'phon': 'plays', 'cat': 'V'}, [('suj', 'W2'), ('obj', 'W5')]),
 'W4': ({'phon': 'the', 'cat': 'DET'}, [('det', 'W4')]),
 'W5': ({'phon': 'fool', 'cat': 'N'}, [])}

In [17]:
def construct_rich_graph(sentence):
    word_list = nltk.word_tokenize(sentence)
    tag_list = nltk.pos_tag(word_list)
    feat_list = [{'phon': n[0], 'cat':n[1]} for n in tag_list]
    t_graph = {f'W{i}': (feat_list[i], []) for i in range(len(tag_list))}
    
    for i in range( len(word_list) - 1): 
        add_edge(t_graph, f'W{i}', 'SUC', f'W{i+1}')
    return t_graph

t_graph = construct_rich_graph("She takes a glass")
t_graph

{'W0': ({'phon': 'She', 'cat': 'PRP'}, [('SUC', 'W1')]),
 'W1': ({'phon': 'takes', 'cat': 'VBZ'}, [('SUC', 'W2')]),
 'W2': ({'phon': 'a', 'cat': 'DT'}, [('SUC', 'W3')]),
 'W3': ({'phon': 'glass', 'cat': 'NN'}, [])}

### 1.3. Information searches

In [18]:
g['W4'][0]

{'phon': 'the', 'cat': 'DET'}

In [19]:
def get_label(graph, node): 
    return graph[node][0]

In [20]:
def get_sucs(graph, node):
    return graph[node][1]

#### 1.3.1. Access to nodes

In [21]:
nodes = g.keys()

In [22]:
verbs = [node for node in nodes if get_label(g, node)['cat'] == "V"]


### EXERCISE 1.3
Find the list of node identiﬁers corresponding to the word
"the" in the graph of "the child plays the fool".

In [23]:
p = construct_rich_graph("the child plays the fool")

the = [node for node in p if get_label(p, node)['phon'] == "the"]

the

['W0', 'W3']

### EXERCISE 1.4. 
How can we ﬁnd out if the same word occurs twice in a
graph?

In [24]:
from collections import Counter

word_freq = Counter([get_label(p, node)['phon'] for node in p])

[c for c in word_freq if word_freq[c] ==2]

['the']

#### 1.3.1. Extracting edges

In [25]:
def make_triplets(g): 
    return [(s,e,t) for s in g for (e,t) in get_sucs(g,s)]

In [28]:
triplets = make_triplets(g)
triplets


[('W2', 'det', 'W1'),
 ('W3', 'suj', 'W2'),
 ('W3', 'obj', 'W5'),
 ('W4', 'det', 'W4')]

In [ ]:
subject_verbs = [ (s , v ) for (v , e , s ) in triplets if e == 'suj']

In [ ]:
def are_related(g,u,v):
    triplets = make_triplets(g)
    for (s,e,t) in triplets:
        print((s,t), (u,v))
        if (s, t) == (u, v): 
            return True
    return False


In [27]:
def is_root(g , u ):
    triplets = make_triplets(g)

    for (s , e , t ) in triplets:
        if t == u:
            return False
    return True

### EXERCISE 1.5.
Rewrite the functions are_related and is_root without
using triplets.

In [75]:
def are_related(graph,u,v):
    for node in graph: 
        sucs = get_sucs(graph, node)
        for suc in sucs: 
            if (node ==u and suc[1] == v) or (node == v and suc[1] == u): 
                return True
    return False 


In [76]:
def is_root(graph , u ):
    for node in graph: 
        sucs = get_sucs(graph, node)
        for suc in sucs: 
            if suc[1] == u: 
                return False
    return True


In [77]:
u = "W3"
v= "W2"
are_related(g, u, v)
for node in g: 
    print(is_root(g, node))

False
False
True
False
False


### EXERCISE 1.6.
A node is known as a leaf if it has no children. Write a
function to ﬁnd out whether a node in a graph is a leaf. Deﬁne a function with
the proﬁle: def is_leaf(g, u).

In [78]:
def is_leaf(g, u): 
    sucs = get_sucs(g, u) 
    print(u, sucs)
    if not sucs: 
        return True
    return False

In [79]:
for i in range(1, 6): 
    print(is_leaf(g, f"W{i}"))

W1 []
True
W2 [('det', 'W1')]
False
W3 [('suj', 'W2'), ('obj', 'W5')]
False
W4 [('det', 'W4')]
False
W5 []
True


### EXERCISE 1.7.

Write a function to select node triplets (s, v, o)
corresponding to the subject-verb-object conﬁguration:

In [80]:

def get_svo(g):
    for node, labels in g.items():
        if is_verb(g, node) and is_root(g, node): 
            suj, obj = get_subj_obj(g, node)
            return suj, node, obj
            
def get_subj_obj(g, v): 
    s, o = "", ""
    for (e, t) in get_sucs(g, v): 
        if e == "suj": 
            s = t
        if e == "obj": 
            o = t
    return s, o
    
def is_verb(g, u): 
    return True if g[u][0]['cat'] == 'V' else False

In [82]:
get_svo(g)

('W2', 'W3', 'W5')

### EXERCISE 1.8.
A graph is said to be linear if it has a root node that only has
one child, which only has one child, and so on up to a leaf node. An example
can be found in exercise 1.1. Write a function to show if a graph is linear.

In [60]:
def get_root(g): 
    for node in g: 
        if is_root(g, node): 
            return node

        
def traverse(g, node):
    children = get_sucs(g, node)
    if len(children) == 1: 
        child = children[0][1]
        if is_leaf(g, child): 
            return True
        else: 
            return traverse(g, child)
    return False


def is_linear(g): 
    root = get_root(g)
    return traverse(g, root)
    

In [62]:
linear_graph = construct_flat_graph("She takes a glass")

In [63]:
is_linear(linear_graph)


W1 [('SUC', 'W2')]
W2 [('SUC', 'W3')]
W3 []


True

### 1.4 Recreating an order


In [67]:
def get_phonology(g):
    def get_idx ( node ) : # gets the float after 'W ' in node if any
        import re # for regular expressions
        word_id = re.search( r'W(\d+(\.\d+)?)', node )
        return word_id.group(1) if word_id else None
    words = { get_idx(node): get_label(g , node )[ 'phon']
             for node in g if get_idx ( node ) }
    return ' '.join([ words [ idx ] for idx in sorted ( words ) ])

get_phonology ( g )

'the child plays the fool'

### EXERCISE 1.9.

Write a function to ﬁnd subject/verb patterns as described
above, but which only retains cases in which the subject occurs before the
verb. For this exercise, we presume that word identiﬁers are arranged in a
way that is compatible with the order of the words themselves (alphabetical
order). The exercise may also be carried out without this presumption,
supposing each node to be connected to its successor by a 'SUC' connection.

In [90]:
def get_idx ( node ) : # gets the float after 'W ' in node if any
    import re # for regular expressions
    word_id = re.search( r'W(\d+(\.\d+)?)', node )
    return word_id.group(1) if word_id else None

def are_ordered(g, first, second): 
    try: 
        return True if get_idx(first) < get_idx(second) else False 
    except NoneType: 
        if ('SUC', second) in get_sucs(g, first): 
            return True
        return False
    
def get_suj_verb_order(g): 
    triplets = make_triplets(g) 
    
    for (s,e,t) in triplets: 
        if is_verb(g, s) and e == "suj" and are_ordered(g, t,s):
            return (t,s)

In [91]:
get_suj_verb_order(g)

('W2', 'W3')

### 1.5. Using patterns with the GREW library

In [3]:
import grew
grew.init()


[Grewpy] Port 8888 already used, failed to open socket
[Grewpy] Port 8889 already used, failed to open socket
connected to port: 8890


<Popen: returncode: None args: ['grewpy', '--caller', '11352', '--port', '88...>

In [4]:
g = grew.graph('''graph {
  W1 [phon="the", cat=DET];
  W2 [phon="child", cat=N];
  W3 [phon="plays", cat=V];
  W4 [phon="the", cat=DET];
  W5 [phon="fool", cat=N];
  W2 -[det]->W1;
  W3 -[suj]->W2;
  W3 -[obj]->W5;
  W5 -[det]->W4;
}''')

In [30]:
grew.search(
"""
pattern {
    N [cat=V ]; M[cat=N]; M -[det]-> D;
} commands {
    
}
""",g
)

[{'nodes': {'N': 'W3', 'M': 'W5', 'D': 'W4'}, 'edges': {}},
 {'nodes': {'N': 'W3', 'M': 'W2', 'D': 'W1'}, 'edges': {}}]

In [5]:
grew.search("pattern { X [cat=V] }", g )

[{'nodes': {'X': 'W3'}, 'edges': {}}]

####  1.5.1 pattern syntax

```python
pattern {
C_1; ... ; C_k;
}
without {
C'_1; ... ; C'_m
}
```

C_x are clauses. There are three types of
clauses: node declarations, edge declarations and additional pattern
constraints.

Node declaration: 
```
N [cat=V, m=ind|subj, t<>fut, n=*, !p, lemma="être"];
``` 

Edge syntax: 
```
N -> M;
N -[suj]-> M;
N -[suj|obj]-> M;
N -[^suj|obj]-> M;
N -[re"aux.*"]-> M;
```

Syntax of regex: https://ocaml.org/api/Str.html

### 1.6 Graph rewriting

In [32]:
r = """rule passiveAgt {
  pattern {
    V [cat=V, m=pastp];
    V -[aux.pass]-> AUX;
    e: V -[suj]-> SUJ;
    P [phon=par]; V -[p_obj.agt]-> P;
    P -[obj.p]-> A;
} commands {
    del_node P;
    del_node AUX;
    add_edge V -[suj]-> A;
    add_edge V -[obj]-> SUJ;
    del_edge e;
} }"""

In [33]:
g = grew.graph('''graph{
  W1 [phon="John",cat=NP];
  W2 [phon="est",cat=V ];
  W3 [phon="mordu", cat=V, m=pastp];
  W4 [phon="par",cat=P ];
  W5 [phon="le", cat=D];
  W6 [word="chien",cat=NP];

  W3 -[suj]-> W1;
  W3 -[aux.pass]-> W2;
  W3 -[p_obj.agt]-> W4;
  W6 -[det]-> W5;
  W4 -[obj.p]-> W6;
}''')

In [34]:
grew.run(r, g, 'passiveAgt')

[{'W6': [{'cat': 'NP', 'word': 'chien'}, [['det', 'W5']]],
  'W5': [{'cat': 'D', 'phon': 'le'}, []],
  'W3': [{'cat': 'V', 'm': 'pastp', 'phon': 'mordu'},
   [['suj', 'W6'], ['obj', 'W1']]],
  'W1': [{'cat': 'NP', 'phon': 'John'}, []]}]

In [51]:
rule = """
rule du2dele {
  pattern {
    A [cat="P+D", phon="du"]; 
    N [cat=N];
    A -[obj.p]-> N;
    }
  commands {
    add_node D:> A; D.cat=D ; D.phon="le" ;
    A.cat=P; A.phon="de";
    add_edge N -[det]-> D;
  }
}
"""
grew.run(rule, g, 'du2dele') 

[]

### 1.6.1. Commands
* feature modification (reassignment, using `=`)
* Node deletion (`del_node`) 
* Node creation ( `add_node` + `:<` or `:>` operators to specify order of new nodes in relation to existing nodes) 
* Edge deletion (`del_edge` + `Node1-[edge_label]->Node2`)
* Edge creation (`add_edge`)
* Edge shifting
    - `shift_in`:Redirect edges that arrive in one node to arrive in another node, 
    - `shift_out`:Redirect eges leaving one node to leave from another node, 
    - `shift` + `Node1 ==> Node2`: modify all edges leaving or entering Node1, shift them onto Node2. 


### 1.6.2 Strategies
A way to specify the order in which rules are applied

In [59]:
sentence1_1 = "La porte du jardin du voisin"

sent_1_1 = grew.graph('''graph{
  W1 [phon=La, cat=D];
  W2 [phon=porte, cat=N];
  W3 [phon=du, cat="P+D"];
  W4 [phon=jardin, cat=N];
  W5 [phon=du, cat="P+D"];
  W6 [phon=voisin, cat=N];
  W2 -[det]-> W1;
  W2 -[dep]-> W3;
  W3 -[obj.p]-> W4;
  W4 -[dep]-> W5;
  W5 -[obj.p]-> W6;
}''')

sentence1_2 = "Le chien du voisin est pris par John"

sent_1_2 = grew.graph('''graph{
  W1 [phon=Le, cat=D];
  W2 [phon=chien, cat=N];
  W3 [phon=du, cat="P+D"];
  W4 [phon=voisin, cat=N];
  W5 [phon=est, cat=V];
  W6 [phon=pris, cat=V, m=pastp];
  W7 [phon=par, cat=P];
  W8 [phon=John, cat=N];
  W2 -[det]-> W1;
  W2 -[dep]-> W3;
  W3 -[obj.p]-> W4;
  W6 -[suj]-> W2;
  W6 -[aux.pass]-> W5;
  W6 -[p_obj.agt]-> W7;
  W7 -[obj.p]-> W8;
}''')

In [57]:
rs =grew.grs("""

""".join([r, rule, "strat S1 { du2dele }"])
)

print(rs)


9


In [64]:
grew.run(rs,sent_1_1,"S1")

[{'_6_': [{'cat': 'D', 'phon': 'le'}, []],
  'W6': [{'cat': 'N', 'phon': 'voisin'}, []],
  'W5': [{'cat': 'P+D', 'phon': 'du'}, [['obj.p', 'W6']]],
  'W4': [{'cat': 'N', 'phon': 'jardin'}, [['det', '_6_'], ['dep', 'W5']]],
  'W3': [{'cat': 'P', 'phon': 'de'}, [['obj.p', 'W4']]],
  'W2': [{'cat': 'N', 'phon': 'porte'}, [['dep', 'W3'], ['det', 'W1']]],
  'W1': [{'cat': 'D', 'phon': 'La'}, []]},
 {'_6_': [{'cat': 'D', 'phon': 'le'}, []],
  'W6': [{'cat': 'N', 'phon': 'voisin'}, [['det', '_6_']]],
  'W5': [{'cat': 'P', 'phon': 'de'}, [['obj.p', 'W6']]],
  'W4': [{'cat': 'N', 'phon': 'jardin'}, [['dep', 'W5']]],
  'W3': [{'cat': 'P+D', 'phon': 'du'}, [['obj.p', 'W4']]],
  'W2': [{'cat': 'N', 'phon': 'porte'}, [['dep', 'W3'], ['det', 'W1']]],
  'W1': [{'cat': 'D', 'phon': 'La'}, []]}]

### 1.6.2.1. Alternative

`Alt(S1, S2)`  where S1 and S2 are two previously defined strategies(or rules). Makes a union of the graph sets obtained by applying S1 or S2. 

In [66]:
grew.run(rs, sent_1_2, 'Alt (passiveAgt, du2dele)')

[{'_8_': [{'cat': 'D', 'phon': 'le'}, []],
  'W8': [{'cat': 'N', 'phon': 'John'}, []],
  'W7': [{'cat': 'P', 'phon': 'par'}, [['obj.p', 'W8']]],
  'W6': [{'cat': 'V', 'm': 'pastp', 'phon': 'pris'},
   [['p_obj.agt', 'W7'], ['aux.pass', 'W5'], ['suj', 'W2']]],
  'W5': [{'cat': 'V', 'phon': 'est'}, []],
  'W4': [{'cat': 'N', 'phon': 'voisin'}, [['det', '_8_']]],
  'W3': [{'cat': 'P', 'phon': 'de'}, [['obj.p', 'W4']]],
  'W2': [{'cat': 'N', 'phon': 'chien'}, [['dep', 'W3'], ['det', 'W1']]],
  'W1': [{'cat': 'D', 'phon': 'Le'}, []]},
 {'W8': [{'cat': 'N', 'phon': 'John'}, []],
  'W6': [{'cat': 'V', 'm': 'pastp', 'phon': 'pris'},
   [['suj', 'W8'], ['obj', 'W2']]],
  'W4': [{'cat': 'N', 'phon': 'voisin'}, []],
  'W3': [{'cat': 'P+D', 'phon': 'du'}, [['obj.p', 'W4']]],
  'W2': [{'cat': 'N', 'phon': 'chien'}, [['dep', 'W3'], ['det', 'W1']]],
  'W1': [{'cat': 'D', 'phon': 'Le'}, []]}]

### 1.6.2.2. Sequence
`Seq(S1, S2)`: produces graphs obtained by successively
applying S1 and S2.

### 1.6.2.3. Pick
`Pick(S1)`

### 1.6.2.4. Iteration
`Iter(S1)` applies S1 as long as it can be applied. Graphs from intermediary stages are omitted.

### 1.6.3. Using lexicons
Assuming a lexicon file called "verb_with_pobjo_noun.lp": 

```
verb prep
...
comparaître devant
comploter contre
compter parmi
compter sur
concorder avec
consister en
contraster avec
...
``` 

In [69]:
lex_rule ="""
rule (lex from "verb_with_pobjo_noun.lp") {
    pattern {
        V [cat=V,lemma=lex.verb]; P [cat=P,lemma=lex.prep]; e: V -[mod]-> P
    }
    commands { del_edge e; add_edge V -[p_obj.o]-> P; }
}
"""

## Utprøving

In [70]:
# Match any node and give it the name N

m1= "pattern { N [] }"

# 